In [4]:
import pandas as pd
import spacy
from collections import Counter
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# spaCy 모델 로드
nlp = spacy.load("en_core_web_sm")

# CSV 파일 읽기 (인코딩 이슈 고려)
df = pd.read_csv("essay.csv", encoding="ISO-8859-1")

# 사람이 쓴 글만 필터링 (label 0)
generated_df = df[df['labels'].astype(str) == "0"].fillna({'text': ''})

# 단어 후보 수집
candidate_words = []

for doc in nlp.pipe(generated_df['text'].astype(str), batch_size=50):
    for token in doc:
        if (
            token.text and
            token.pos_ in {"ADV", "SCONJ", "INTJ"} and
            token.text[0].isupper() and
            token.is_alpha and
            not token.is_stop
        ):
            candidate_words.append(token.text)

# 빈도수 계산
word_counts = Counter(candidate_words)
most_common = word_counts.most_common(100)

# 결과 저장
result_df = pd.DataFrame(most_common, columns=["word", "count"])
result_df.index += 1
result_df.index.name = "rank"

# 출력 및 CSV 저장
print(result_df)
result_df.to_csv("1_human.csv", index=True)


Saving essay.csv to essay.csv
             word  count
rank                    
1         Finally    976
2          Lastly    882
3             Yes    691
4           Maybe    616
5     Furthermore    547
...           ...    ...
96      Seriously      8
97       Suddenly      8
98        Earlier      8
99       Foremost      7
100     Naturally      7

[100 rows x 2 columns]


In [7]:
import pandas as pd
import spacy
from collections import Counter

# 1. spaCy 모델 로드
nlp = spacy.load("en_core_web_sm")

# 2. 업로드된 파일 그대로 사용
df = pd.read_csv("essay.csv", encoding="ISO-8859-1")

# 3. AI가 쓴 글만 필터링 (label = 1)
generated_df = df[df['labels'].astype(str) == "1"].fillna({'text': ''})

# 4. 샘플 3000개만 추출 + 500자 이하로 자르기 (속도 최적화)
sample_df = generated_df.sample(3000, random_state=42)
sample_df['text'] = sample_df['text'].str.slice(0, 500)

# 5. 단어 후보 수집
candidate_words = []

for doc in nlp.pipe(sample_df['text'].astype(str), batch_size=100):
    for token in doc:
        if (
            token.text and
            token.pos_ in {"ADV", "SCONJ", "INTJ"} and
            token.text[0].isupper() and
            token.is_alpha and
            not token.is_stop
        ):
            candidate_words.append(token.text)

# 6. 빈도수 계산
word_counts = Counter(candidate_words)
most_common = word_counts.most_common(100)

# 7. 결과 저장
result_df = pd.DataFrame(most_common, columns=["word", "count"])
result_df.index += 1
result_df.index.name = "rank"

# 8. 출력 및 저장
print(result_df)
result_df.to_csv("1_ai_filtered.csv", index=True)


              word  count
rank                     
1          Firstly    237
2     Additionally    133
3          Despite     61
4      Furthermore     50
5           Second     35
...            ...    ...
58       Basically      1
59          Whilst      1
60    Economically      1
61       Generally      1
62          Simply      1

[62 rows x 2 columns]


In [12]:
import pandas as pd
import spacy
from collections import Counter

# spaCy 모델 로드
nlp = spacy.load("en_core_web_sm")

# 이미 업로드된 essay.csv 파일 사용
df = pd.read_csv("essay.csv", encoding="ISO-8859-1")

# ✅ AI가 쓴 글 필터링 (label = 1)
generated_df = df[df['labels'].astype(str) == "1"].fillna({'text': ''})

# 단어 수집 리스트 초기화
candidate_words = []

# ✅ 대소문자 무시 + POS + 불용어 제거
for doc in nlp.pipe(generated_df['text'].astype(str), batch_size=100):
    for token in doc:
        if (
            token.text and
            token.pos_ in {"ADV", "SCONJ", "INTJ"} and
            token.is_alpha and
            not token.is_stop
        ):
            candidate_words.append(token.text.lower())

# 빈도 계산 및 저장
word_counts = Counter(candidate_words)
most_common = word_counts.most_common(100)

result_df = pd.DataFrame(most_common, columns=["word", "count"])
result_df.index += 1
result_df.index.name = "rank"

print(result_df)
result_df.to_csv("2_ai.csv", index=True)


              word  count
rank                     
1     additionally   3861
2      furthermore   2804
3          finally   2632
4       ultimately   1805
5          instead   1763
...            ...    ...
96       extremely    189
97          firmly    186
98        outdoors    185
99         clearly    181
100    responsibly    179

[100 rows x 2 columns]


In [14]:
import pandas as pd
import spacy
from collections import Counter

# spaCy 모델 로드
nlp = spacy.load("en_core_web_sm")

# CSV 불러오기 (이미 업로드된 essay.csv 사용)
df = pd.read_csv("essay.csv", encoding="ISO-8859-1")

# ✅ 사람이 쓴 텍스트만 필터링 (label = 0)
generated_df = df[df['labels'].astype(str) == "0"].fillna({'text': ''})

# 단어 리스트 초기화
candidate_words = []

# ✅ 모든 품사 대상 + 알파벳 단어 + 불용어 제거 + 소문자 통일
for doc in nlp.pipe(generated_df['text'].astype(str), batch_size=100):
    for token in doc:
        if (
            token.text and
            token.is_alpha and
            not token.is_stop
        ):
            candidate_words.append(token.text.lower())

# ✅ 빈도수 계산
word_counts = Counter(candidate_words)
most_common = word_counts.most_common(100)

# ✅ 결과 저장
result_df = pd.DataFrame(most_common, columns=["word", "count"])
result_df.index += 1
result_df.index.name = "rank"

print(result_df)
result_df.to_csv("3_human.csv", index=True)


           word  count
rank                  
1        people  58505
2           car  24200
3          like  23906
4     electoral  23782
5         venus  22479
...         ...    ...
96       united   4421
97         says   4411
98      reasons   4402
99    candidate   4352
100        find   4273

[100 rows x 2 columns]


In [15]:
import pandas as pd
import spacy
from collections import Counter

# spaCy 모델 로드
nlp = spacy.load("en_core_web_sm")

# CSV 불러오기 (이미 업로드된 essay.csv 사용)
df = pd.read_csv("essay.csv", encoding="ISO-8859-1")

# ✅ AI가 쓴 텍스트만 필터링 (label = 1)
generated_df = df[df['labels'].astype(str) == "1"].fillna({'text': ''})

# 단어 리스트 초기화
candidate_words = []

# ✅ 모든 품사 대상 + 알파벳 단어 + 불용어 제거 + 소문자 통일
for doc in nlp.pipe(generated_df['text'].astype(str), batch_size=100):
    for token in doc:
        if (
            token.text and
            token.is_alpha and
            not token.is_stop
        ):
            candidate_words.append(token.text.lower())

# ✅ 빈도수 계산
word_counts = Counter(candidate_words)
most_common = word_counts.most_common(100)

# ✅ 결과 저장
result_df = pd.DataFrame(most_common, columns=["word", "count"])
result_df.index += 1
result_df.index.name = "rank"

print(result_df)
result_df.to_csv("3_ai.csv", index=True)


               word  count
rank                      
1            people  17171
2          students  11529
3         electoral  11523
4           college  10851
5         important   8996
...             ...    ...
96           access   2475
97          quality   2455
98            ideas   2433
99    opportunities   2404
100          reduce   2381

[100 rows x 2 columns]
